In [1]:
import moviepy.editor as mp
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.io.VideoFileClip import VideoFileClip
import speech_recognition as sr
from tkinter.filedialog import askopenfilename
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import json

In [8]:
name_file = "Javi"
n_file = 5

In [35]:
# Extraigo el audio del vídeo

video = mp.VideoFileClip(".\\videos\\{}_{}.mp4".format(name_file, n_file))
audio = video.audio.write_audiofile(r".\\audios\\{}_{}.wav".format(name_file, n_file)) # de cuando se recibia un vídeo
# HASTA AQUÍ SI SE UTILIZA EL GOOGLE CLOUD SPEECH TO TEXT API
recibido_cambio = sr.Recognizer()
speech_audio = sr.AudioFile(".\\audios\\{}_{}.wav".format(name_file, n_file))

chunk:   0%|          | 0/1365 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in .\\audios\\Javi_5.wav


MoviePy - Done.


In [24]:
# Reconocedor básico (ANTIGUO)

with speech_audio as fuente:
    #Se reduce el ruido
    recibido_cambio.adjust_for_ambient_noise(fuente)
    audio = recibido_cambio.record(fuente)
    #Transcripcion
    texto_transcrito = recibido_cambio.recognize_google(audio, language = "es-ES")
    print(texto_transcrito)

matrículas celador aproximó rotonda rotonda media levanto el pie del acelerador rotonda cerca izquierda libre intermitente izquierda entre la rotonda gira izquierda acelero quita el intermitente intermitente derecha giro derecha salgo de rotonda sigue recto


In [39]:
# RECONOCEDOR DE GOOGLE CLOUD - SPEECH TO TEXT API

import os
from google.cloud import speech

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'key.json'
speech_client = speech.SpeechClient()

audio_filename = ".\\audios\\{}_{}.wav".format(name_file, n_file)
with open(audio_filename, 'rb') as f:
    byte_data_wav = f.read()

audio_wav = speech.RecognitionAudio(content=byte_data_wav)
config_wav = speech.RecognitionConfig(
    sample_rate_hertz = 44100, # el sample ratio de hercios para los archivos WAV
    language_code = 'es-ES', # código de idioma para español de España
    enable_word_time_offsets = True, # tag para obtener los timestamps de las palabras
    audio_channel_count = 2
)

# Aquí se usa el reconocedor para devolver un JSON con la transcripción generada
# Esto es para cuando el archivo de audio dura menos de 1 minuto y pesa menos de 10Mb
response_wav = speech_client.recognize(
    config = config_wav,
    audio = audio_wav
)

# Esto sirve para cuando el archivo supera el minuto y las 10Mb de tamaño
'''media_uri = 'gs://tfg-cdr/Javi_5.wav'
long_audio_wav = speech.RecognitionAudio(uri=media_uri)
long_config_wav = speech.RecognitionConfig(
    sample_rate_hertz = 44100,
    language_code = 'es-ES',
    enable_word_time_offsets = True,
    audio_channel_count = 2
)

long_response_wav = speech_client.long_running_recognize(
    config = long_config_wav,
    audio = long_audio_wav
)

response_wav = long_response_wav.result(timeout=90)'''

# Se abre un JSON nuevo y se carga en la variable "data"
json_filename = ".\\t_raw\\{}_{}.json".format(name_file, n_file)
with open(json_filename, "w", encoding='utf-8') as f:
    data = json.dump([], f)
with open(json_filename, "r", encoding='utf-8') as f:
    data = json.load(f)

# Cargo la lista de stopwords para el español
sw = set(stopwords.words("spanish"))

# Navego por el JSON resultado de la transcripción
for result in response_wav.results:
    alternative = result.alternatives[0]
    for word_info in alternative.words:
        word = word_info.word
        word = word.lower()
        start_time = word_info.start_time.total_seconds()
        # Elimino ya las palabras que sean stopwords excepto la palabra "no"
        if word not in sw or word == 'no':
            # Genero un pequeño diccionario a modo de objeto para guardar la información de la palabra y su start time
            entry = {
                'word': word,
                'start': start_time
            }
            # Lo meto en la lista "data"
            data.append(entry)

# Guardo el contenido de "data" en el archivo JSON para revisarlo
with open(json_filename, "w", encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False)

In [42]:
# Estemización de las palabras transcritas desde la API de GOOGLE CLOUD SPEECH TO TEXT

json_filename = ".\\t_processed\\{}_{}.json".format(name_file, n_file)
with open(json_filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

json_filename_rev = ".\\t_processed\\{}_{}_rev.json".format(name_file, n_file)
with open(json_filename_rev, "w", encoding='utf-8') as f:
    data_rev = json.dump([], f)
with open(json_filename_rev, "r", encoding='utf-8') as f:
    data_rev = json.load(f)

for t in data:
    word = t['word']
    start = t['start']
    entry = {
        'word': word,
        'stem': SnowballStemmer('spanish').stem(word),
        'start': start
    }
    data_rev.append(entry)

with open(json_filename_rev, "w", encoding='utf-8') as f:
    json.dump(data_rev, f, ensure_ascii=False)

In [65]:
# Guardo el texto en bruto transcrito de los audios (ESTO ES CON EL RECONOCEDOR ANTIGUO)

filename_t_raw = ".\\t_raw\\{}_{}.txt".format(name_file, n_file)
with open(filename_t_raw, "w", encoding="utf-8") as f:
    f.write(texto_transcrito)
print("Se ha guardado la transcripción en el archivo " + filename_t_raw)

Se ha guardado la transcripción en el archivo .\t_raw\Borja_7.txt


In [ ]:
# PARSEO PARA LOS DATOS DEL RECONOCEDOR BÁSICO

f = open ('.\\t_processed\\{}_{}.txt'.format(name_file, n_file),'r',encoding='utf8')
text = f.read()

sw = set(stopwords.words("spanish"))
# Limpio con regex el texto
text = re.sub('[%s]' % re.escape(string.punctuation + "'" + '"' + "’" + '”' + '“' + "•‘"), ' ', str(text))
text = re.sub('\w*\d\w*', ' ', str(text))
# Pongo el texto en minúsculas
text = text.lower()
# Tokenizo por palabras
text = word_tokenize(str(text), language='spanish')
tokens = []
for t in text:
    if not t in sw or t == 'no':
        tokens.append(t)

json_filename = ".\\t_processed\\{}_{}.json".format(name_file, n_file)
with open(json_filename, "w", encoding='utf-8') as f:
    data = json.dump([], f)
with open(json_filename, "r", encoding='utf-8') as f:
    data = json.load(f)
for t in tokens:
    entry = {
        'word': t,
        'stem': SnowballStemmer('spanish').stem(t),
        'start': 0.0
    }
    data.append(entry)

with open(json_filename, "w", encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False)

In [43]:
# Se añaden los tags y se genera el JSON de PVA completo

instrucciones = {
    'aceler': 'T-ON',
    'acerc rotond': 'APROX',
    'rotond medi': 'RND-MD',
    'manteng aceler': 'T-HOLD',
    'rotond cerc': 'RND-NR',
    'izquierd libr': 'L-FREE',
    'izquierd ocup': 'L-BUSY',
    'fren ced': 'B-ON',
    'fren stop': 'B-ON',
    'fren': 'B-ON',
    'stop': 'B-ON',
    'ced': 'RND-IN',
    'entro rotond': 'RND-IN',
    'gir derech': 'TURN-R',
    'intermitent izquierd': 'LB-ON',
    'pong intermitent izquierd': 'LB-ON',
    'intermitent izquierd rotond': 'LB-ON',
    'rotond gir izquierd': 'TURN-L',
    'rotond gir derech': 'TURN-R',
    'gir izquierd': 'TURN-L',
    'quit intermitent': 'BLK-OFF',
    'intermitent derech': 'RB-ON',
    'pong intermitent derech': 'RB-ON',
    'intermitent derech rotond': 'RB-ON',
    'salg rotond': 'RND-EXIT',
    'aproxim rotond': 'APROX',
    'levant aceler': 'T-OFF',
    'levant pie': 'T-OFF',
    'levant pie aceler': 'T-OFF',
    'no vien nadi': 'RND-CLEAR',
    'gir izquierd rotond': 'TURN-L',
    'gir derech rotond': 'TURN-R',
    'atencion ciclist': 'CAR-NR',
    'atencion coche': 'CAR-NR',
    'centr libr': 'L-FREE',
    'centr ocup': 'F-BUSY',
    'enderez': 'TURN-STR',
    'sig rect': 'STR',
    'rect': 'STR'
}

# Utilizar este json_filename si se ha transcrito con el reconocedor básico
#json_filename = ".\\t_processed\\{}_{}.json".format(name_file, n_file)

# Utilizar este json_filename si se ha transcrito con el reconocedor de la API de GOOGLE CLOUD SPEECH TO TEXT
json_filename = ".\\t_processed\\{}_{}_rev.json".format(name_file, n_file)

with open(json_filename, "r", encoding='utf-8') as f:
    data = json.load(f)

dataset = {}
count = 0

for i, x in enumerate(data):
    w = x['stem']
    if (len(data) > i + 2) and x['stem'] + ' ' + data[i + 1]['stem'] + ' ' + data[i + 2]['stem'] in instrucciones:
        sentence = x['word'] + ' ' + data[i + 1]['word'] + ' ' + data[i + 2]['word']
        sentence_t = x['stem'] + ' ' + data[i + 1]['stem'] + ' ' + data[i + 2]['stem']
        dataset["inst_{}".format(count)] = [x, data[i + 1], data[i + 2]]
        dataset["inst_{}".format(count)].append({"tag": instrucciones[sentence_t], "sentence": sentence})
        count += 1
        del data[i + 1]
        del data[i + 1]
    elif (len(data) > i + 1) and x['stem'] + ' ' + data[i + 1]['stem'] in instrucciones:
        sentence = x['word'] + ' ' + data[i + 1]['word']
        sentence_t = x['stem'] + ' ' + data[i + 1]['stem']
        dataset["inst_{}".format(count)] = [x, data[i + 1]]
        dataset["inst_{}".format(count)].append({"tag": instrucciones[sentence_t], "sentence": sentence})
        count += 1
        del data[i + 1]
    elif w in instrucciones:
        dataset["inst_{}".format(count)] = [x]
        dataset["inst_{}".format(count)].append({"tag": instrucciones[w], "sentence": x['word']})
        count += 1

json_out = ".\\t_tags\\{}_{}.json".format(name_file, n_file)
with open(json_out, "w", encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False)

In [9]:
# Fusión del JSON de pensamiento en voz alta con el JSON de Carla correspondiente a cada grabación

pva_json_filename = ".\\t_tags\\{}_{}.json".format(name_file, n_file)
with open(pva_json_filename, 'r', encoding='utf-8') as f:
    pva_data = json.load(f)

carla_json_filename = ".\\dataset\\parse\\{}_{}\\SIM_DATA.json".format(name_file, n_file)
with open(carla_json_filename, 'r', encoding='utf-8') as f:
    carla_data = json.load(f)

for inst in pva_data:
    start = pva_data[inst][0]['start']
    last = pva_data[inst][-1]
    for i, tick in enumerate(carla_data):
        if (i + 1) < len(carla_data):
            tick_next = carla_data[i + 1]
            if tick['timestamp'] <= start and tick_next['timestamp'] > start:
                carla_dic = dict(tick)
                last['carla'] = carla_dic
                light_state = last['carla']['control']['light_state']
                light_state = "Left_Blinker" if light_state == 32 else "Right_Blinker" if light_state == 16 else "NONE"
                last['carla']['control']['light_state'] = light_state
                del last['carla']['timestamp']
                break
            
final_json_filename = ".\\t_merged\\{}_{}.json".format(name_file, n_file)
with open(final_json_filename, 'w', encoding='utf-8') as f:
    json.dump(pva_data, f, ensure_ascii=False)

In [29]:
# FIX START TIMESTAMPS (30 divisiones a 100 divisiones)

json_filename = ".\\t_tags\\Borja_7.json"
with open(json_filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

for inst in data:
    for a in data[inst]:
        if 'start' in a:
            start = a['start']
            decimal_part = start % 1
            start -= decimal_part
            decimal_part = decimal_part * 100 / 30
            start += decimal_part
            a['start'] = round(start, 2)

with open(json_filename, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False)